In [11]:
import warnings
warnings.filterwarnings('ignore')
# import geoplot as gplt

import geopandas as gpd
# import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import plotly.offline as go_offline
import plotly.graph_objects as go
from shapely.geometry import Point
import mapclassify as mc
import numpy as np
import laspy
import rasterio
from rasterio import mask
import folium

In [3]:
import numpy as np
import plotly.offline as go_offline
import plotly.graph_objects as go

In [5]:
pan = pd.read_csv('../csv/iow.csv')
pan.head()

,Classification,EdgeOfFlightLine,GpsTime,Intensity,NumberOfReturns,OriginId,PointSourceId,ReturnNumber,ScanAngleRank,ScanDirectionFlag,UserData,X,Y,Z
0,2.0,0.0,56160.044,66.0,1.0,17584.0,85.0,1.0,11.0,0.0,69.0,446171.119,4654059.969,283.56
1,2.0,0.0,56160.044,63.0,1.0,17584.0,85.0,1.0,11.0,0.0,69.0,446179.797,4654059.081,281.70
2,2.0,0.0,56160.044,64.0,1.0,17584.0,85.0,1.0,11.0,0.0,69.0,446181.543,4654058.912,281.13
3,2.0,0.0,56160.061,72.0,2.0,17584.0,85.0,2.0,11.0,1.0,69.0,446183.553,4654057.289,281.30
4,2.0,0.0,56160.061,34.0,1.0,17584.0,85.0,1.0,11.0,1.0,69.0,446176.643,4654058.001,282.68


In [14]:

def geometry (csv : str): 
    
    print ('===================READING CSV  =====================')
    pan = pd.read_csv(csv)
    
    geo = pan[["X","Y","Z"]]
    
    #convert xyz to geodataframe
    geometry = [Point(xy) for xy in zip(pd.to_numeric(gp['X']), pd.to_numeric(gp['Y']))]
    gdf = gpd.GeoDataFrame(gp, crs='epsg:4326',geometry=geometry)
    gdf = gdf[["Z", "geometry"]]
    gdf = gdf.rename(columns={"Z": "elevation_m", "geometry": "geometry"})
    #gdf.head()
    
    return gdf

In [13]:
geometry('../csv/iow.csv')

===================READING CSV  =====================


,elevation_m,geometry
0,283.56,POINT (446171.119 4654059.969)
1,281.70,POINT (446179.797 4654059.081)
2,281.13,POINT (446181.543 4654058.912)
3,281.30,POINT (446183.553 4654057.289)
4,282.68,POINT (446176.643 4654058.001)
...,...,...
1399616,273.12,POINT (447500.229 4654048.369)
1399617,273.09,POINT (447505.651 4654049.470)
1399618,273.08,POINT (447507.438 4654049.827)
1399619,273.09,POINT (447503.067 4654050.007)


In [11]:
subset = pan.sample(n=1000)
subset

,Classification,EdgeOfFlightLine,GpsTime,Intensity,NumberOfReturns,OriginId,PointSourceId,ReturnNumber,ScanAngleRank,ScanDirectionFlag,UserData,X,Y,Z
572787,2.0,0.0,56162.953,46.0,1.0,17583.0,85.0,1.0,11.0,1.0,69.0,446200.048,4653839.208,276.61
1293811,2.0,0.0,56171.505,73.0,1.0,17583.0,85.0,1.0,7.0,0.0,69.0,446376.767,4653213.636,289.79
747770,2.0,0.0,56170.415,80.0,1.0,17583.0,85.0,1.0,-3.0,1.0,69.0,446857.029,4653222.487,286.24
1186199,2.0,0.0,55099.026,147.0,1.0,17583.0,79.0,1.0,-5.0,0.0,70.0,447370.041,4652636.831,271.77
662336,2.0,0.0,56178.807,61.0,1.0,17583.0,85.0,1.0,-7.0,0.0,69.0,446603.508,4652647.760,279.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1174574,2.0,0.0,55112.869,101.0,1.0,17583.0,79.0,1.0,1.0,1.0,70.0,447606.870,4653536.250,271.89
647049,2.0,0.0,56176.850,156.0,1.0,17583.0,85.0,1.0,-4.0,0.0,69.0,446457.632,4652808.969,289.12
430492,2.0,0.0,56166.220,77.0,1.0,17583.0,85.0,1.0,-18.0,0.0,69.0,447032.994,4653504.622,279.57
747240,2.0,0.0,56170.306,67.0,1.0,17583.0,85.0,1.0,0.0,1.0,69.0,446719.014,4653248.507,294.28


In [ ]:

#Form a new data frame (named cleanTweet), containing columns  𝐜𝐥𝐞𝐚𝐧-𝐭𝐞𝐱𝐭  and  𝐩𝐨𝐥𝐚𝐫𝐢𝐭𝐲 
clean_tweet= ["X","Y","Z"]
CleanTweets = subset[clean_tweet]


,X,Y,Z
572787,446200.048,4653839.208,276.61
1293811,446376.767,4653213.636,289.79
747770,446857.029,4653222.487,286.24
1186199,447370.041,4652636.831,271.77
662336,446603.508,4652647.760,279.64


In [ ]:
geometry = [Point(xyz) for xyz in zip(inFile.x,inFile.y,inFile.z)]
lidar_geodf = GeoDataFrame(lidar_df, crs=crs, geometry=geometry)
lidar_geodf.crs = {'init' :'epsg:2959'} # set correct spatial reference

In [23]:
geo = pd.read_csv('../scripts/data.csv')
geo.head()

,Unnamed: 0,X,Y,Z
0,572787,446200.048,4653839.208,276.61
1,1293811,446376.767,4653213.636,289.79
2,747770,446857.029,4653222.487,286.24
3,1186199,447370.041,4652636.831,271.77
4,662336,446603.508,4652647.760,279.64


In [25]:
from shapely.geometry import Point

In [27]:
geometry = [Point(xy) for xy in zip(pd.to_numeric(geo['X']), pd.to_numeric(geo['Y']))]
gdf = gpd.GeoDataFrame(geo, crs='epsg:4326',geometry=geometry)
gdf = gdf[["Z", "geometry"]]
gdf = gdf.rename(columns={"Z": "elevation_m", "geometry": "geometry"})
gdf.head()

,elevation_m,geometry
0,276.61,POINT (446200.048 4653839.208)
1,289.79,POINT (446376.767 4653213.636)
2,286.24,POINT (446857.029 4653222.487)
3,271.77,POINT (447370.041 4652636.831)
4,279.64,POINT (446603.508 4652647.760)


In [15]:
CleanTweets.to_csv('../scripts/data.csv')

In [28]:
#READING AND PARSING THE DATA
file=open('../scripts/iow.csv')
lines=file.readlines()
n_line=len(lines)
x=[]
y=[]
z=[]
for i in range(1,n_line):
    split_line=lines[i].split(",")
    xyz_t=[]
    x.append(float(split_line[0].rstrip()))
    y.append(float(split_line[1].rstrip()))
    z.append(float(split_line[2].rstrip()))

In [29]:
#DISTANCE FUNCTION
def distance(x1,y1,x2,y2):
    d=np.sqrt((x1-x2)**2+(y1-y2)**2)
    return d

#CREATING IDW FUNCTION
def idw_npoint(xz,yz,n_point,p):
    r=10 #block radius iteration distance
    nf=0
    while nf<=n_point: #will stop when np reaching at least n_point
        x_block=[]
        y_block=[]
        z_block=[]
        r +=10 # add 10 unit each iteration
        xr_min=xz-r
        xr_max=xz+r
        yr_min=yz-r
        yr_max=yz+r
        for i in range(len(x)):
            # condition to test if a point is within the block
            if ((x[i]>=xr_min and x[i]<=xr_max) and (y[i]>=yr_min and y[i]<=yr_max)):
                x_block.append(x[i])
                y_block.append(y[i])
                z_block.append(z[i])
        nf=len(x_block) #calculate number of point in the block
    
    #calculate weight based on distance and p value
    w_list=[]
    for j in range(len(x_block)):
        d=distance(xz,yz,x_block[j],y_block[j])
        if d>0:
            w=1/(d**p)
            w_list.append(w)
            z0=0
        else:
            w_list.append(0) #if meet this condition, it means d<=0, weight is set to 0
    
    #check if there is 0 in weight list
    w_check=0 in w_list
    if w_check==True:
        idx=w_list.index(0) # find index for weight=0
        z_idw=z_block[idx] # set the value to the current sample value
    else:
        wt=np.transpose(w_list)
        z_idw=np.dot(z_block,wt)/sum(w_list) # idw calculation using dot product
    return z_idw

In [21]:
# POPULATE INTERPOLATION POINTS
n=100 #number of interpolation point for x and y axis
x_min=min(x)
x_max=max(x)
y_min=min(y)
y_max=max(y)
w=x_max-x_min #width
h=y_max-y_min #length
wn=w/n #x interval
hn=h/n #y interval

#list to store interpolation point and elevation
y_init=y_min
x_init=x_min
x_idw_list=[]
y_idw_list=[]
z_head=[]
for i in range(n):
    xz=x_init+wn*i
    yz=y_init+hn*i
    y_idw_list.append(yz)
    x_idw_list.append(xz)
    z_idw_list=[]
    for j in range(n):
        xz=x_init+wn*j
        z_idw=idw_npoint(xz,yz,5,1.5) #min. point=5, p=1.5
        z_idw_list.append(z_idw)
    z_head.append(z_idw_list)

In [22]:
# CREATING 3D TERRAIN MODEL
fig=go.Figure()
fig.add_trace(go.Surface(z=z_head,x=x_idw_list,y=y_idw_list))
fig.update_layout(scene=dict(aspectratio=dict(x=2, y=2, z=0.5),xaxis = dict(range=[x_min,x_max],),yaxis = dict(range=[y_min,y_max])))
go_offline.plot(fig,filename='../scripts/3d_terrain.html',validate=True, auto_open=False)

'../scripts/3d_terrain.html'

In [40]:
# get shp from tif
from glob import glob
def get_shp_from_tif(tif_path:str, shp_file_path:str) -> None:
    raster = rasterio.open(tif_path)
    bounds = raster.bounds
    counter=0
    
        
    df = gpd.GeoDataFrame({"geometry":[box(*bounds)]})
   
    # save to file
#     df.to_file(shp_file_path)
    print(df)
    print('Saved..')

In [41]:

from shapely.geometry import box
get_shp_from_tif('../tif/USGS_LPC_CO_SoPlatteRiver_Lot5_2013_LAS_2015.tif', shp_file_path='../shp/io.shp')

#3DEP/USGS_3DEP_LIDAR_Challenge/tif/USGS_LPC_CO_SoPlatteRiver_Lot5_2013_LAS_2015.tif

                                            geometry
0  POLYGON ((-11666600.700 4776607.300, -11666600...
Saved..


In [50]:
import rasterio
from rasterio.features import shapes
mask = None
with rasterio.Env():
    with rasterio.open('../tif/iowa.tif') as src:
        image = src.read(1) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))

In [51]:
list(results)
#  geoms = list(results)
#  # first feature
#  print (geoms[0])

ValueError: image dtype must be one of: int16, int32, uint8, uint16, float32

In [1]:
!pip install gdal


In [2]:
import gdal
raster=gdal.open('../tif/iowa.tif')

ModuleNotFoundError: No module named 'gdal'